In [24]:
from SimpleUGEN import SimpleUGen
import tensorflow as tf
import numpy as np
import tensorflow.keras as k
import tensorflow.keras.backend as K
from tensorflow.keras.optimizers import Adadelta, RMSprop,SGD,Adam

from tensorflow.python.client import device_lib

sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))

K.set_image_data_format('channels_first');

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce RTX 2070 SUPER, pci bus id: 0000:41:00.0, compute capability: 7.5



## Load Data

In [7]:
data = np.load(r"../TrainingData/data.npz")
x_train = data['x_train']
y_train = data['y_train']
print(x_train.shape,y_train.shape)
x_test = data['x_test']
y_test = data['y_test']
print(x_test.shape,y_test.shape)
#del data

(2175, 5, 8, 128, 128) (2175, 1, 8, 128, 128)
(596, 5, 8, 128, 128) (596, 1, 8, 128, 128)


## Make Model

In [25]:
ae = SimpleUGen(do_batch_norm=True)
ae.compile(loss='mean_squared_error', optimizer = RMSprop())
ae.build([200,5,8,128,128])
ae.summary()

(200, 5, 8, 64, 64)
(200, 5, 8, 64, 64)
(200, 10, 8, 32, 32)
(200, 10, 8, 32, 32)
(200, 15, 4, 16, 16)
(200, 15, 4, 16, 16)
(200, 20, 2, 8, 8)
(200, 20, 2, 8, 8)
Model: "SimpleU"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Conv3D_0 (Conv3D)            multiple                  680       
_________________________________________________________________
Conv3D_1 (Conv3D)            multiple                  1360      
_________________________________________________________________
Conv3D_2 (Conv3D)            multiple                  4065      
_________________________________________________________________
Conv3D_3 (Conv3D)            multiple                  8120      
_________________________________________________________________
sequential_45 (Sequential)   (200, 5, 8, 64, 64)       164096    
_________________________________________________________________
sequential_46 (Sequential)   

## Train UGEN

In [26]:
print(x_train.shape)
ae.fit(x_train,y_train,epochs=50,batch_size=50,shuffle=True,validation_data=(x_test, y_test))

(2175, 5, 8, 128, 128)
Epoch 1/50
43/44 [============================>.] - ETA: 0s - loss: 12555.0235